In [ ]:
print("Hllo")

Hllo


In [ ]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 34.9 MB/s eta 0:00:00


In [ ]:
import mysql.connector
from mysql.connector import Error

hostname = "2fydd.h.filess.io"
database = "olistproject_pigbasemet"
port = "3307"
username = "olistproject_pigbasemet"
password = "cc50138b86496b619eb2ae5e34616ac23408e24a"

try:
    connection = mysql.connector.connect(host=hostname, database=database, user=username, password=password, port=port)
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")



<ipython-input-10-98779340382d>:13: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_Info = connection.get_server_info()


Connected to MySQL Server version  8.0.36-28
You're connected to database:  ('olistproject_pigbasemet',)
MySQL connection is closed


In [ ]:
import pandas as pd

order_payments=pd.read_csv("/content/sample_data/olist_order_payments_dataset.csv")
order_payments.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/olist_order_payments_dataset.csv'

In [ ]:
order_payments.shape

In [ ]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

# Connection details
hostname = "2fydd.h.filess.io"
database = "olistproject_pigbasemet"
port = "3307"
username = "olistproject_pigbasemet"
password = "cc50138b86496b619eb2ae5e34616ac23408e24a"

# CSV file path
csv_file_path = "/content/sample_data/olist_order_payments_dataset.csv"

# Table name where the data will be uploaded
table_name = "olist_order_payments"

try:
    # Step 1: Establish a connection to MySQL server
    connection = mysql.connector.connect(
        host=hostname,
        database=database,
        user=username,
        password=password,
        port=port
    )
    if connection.is_connected():
        print("Connected to MySQL Server successfully!")

        # Step 2: Create a cursor to execute SQL queries
        cursor = connection.cursor()

        # Step 3: Drop table if it already exists (for clean insertion)
        cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
        print(f"Table `{table_name}` dropped if it existed.")

        # Step 4: Create a table structure to match CSV file
        create_table_query = f"""
        CREATE TABLE {table_name} (
            order_id VARCHAR(50),
            payment_sequential INT,
            payment_type VARCHAR(20),
            payment_installments INT,
            payment_value FLOAT
        );
        """
        cursor.execute(create_table_query)
        print(f"Table `{table_name}` created successfully!")

        # Step 5: Load the CSV data into pandas DataFrame
        data = pd.read_csv(csv_file_path)
        print("CSV data loaded into pandas DataFrame.")

        # Step 6: Insert data in batches of 500 records
        batch_size = 500  # Define the batch size
        total_records = len(data)  # Get total records in the DataFrame

        print(f"Starting data insertion into `{table_name}` in batches of {batch_size} records.")
        for start in range(0, total_records, batch_size):
            end = start + batch_size
            batch = data.iloc[start:end]  # Get the current batch of records

            # Convert batch to list of tuples for MySQL insertion
            batch_records = [
                tuple(row) for row in batch.itertuples(index=False, name=None)
            ]

            # Prepare the INSERT query
            insert_query = f"""
            INSERT INTO {table_name}
            (order_id, payment_sequential, payment_type, payment_installments, payment_value)
            VALUES (%s, %s, %s, %s, %s);
            """

            # Execute the insertion query for the batch
            cursor.executemany(insert_query, batch_records)
            connection.commit()  # Commit after each batch
            print(f"Inserted records {start + 1} to {min(end, total_records)} successfully.")

        print(f"All {total_records} records inserted successfully into `{table_name}`.")

except Error as e:
    # Step 7: Handle any errors
    print("Error while connecting to MySQL or inserting data:", e)

finally:
    # Step 8: Close the cursor and connection
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed.")


In [ ]:
!pip install pymongo

In [ ]:
# importing module
from pymongo import MongoClient

hostname = "xdtqr.h.filess.io"
database = "olistDataNoSQL_teacherbox"
port = "61004"
username = "olistDataNoSQL_teacherbox"
password = "fcec05230340cf66a26f410a35c9fdaae8852143"

uri = "mongodb://" + username + ":" + password + "@" + hostname + ":" + port + "/" + database

# Connect with the portnumber and host
client = MongoClient(uri)

# Access database
mydatabase = client[database]


In [ ]:
# prompt: read the product_category.csv and create a collection and upload to above mangoDb

# CSV file path
csv_file_path = "/content/sample_data/product_category_name_translation.csv"

# Collection name where the data will be uploaded
collection_name = "product_category"

try:
    # Step 1: Access the database
    mydatabase = client[database]
    print(f"Accessed database: {database}")

    # Step 2: Access or create a collection
    mycollection = mydatabase[collection_name]
    print(f"Accessed or created collection: {collection_name}")

    # Step 3: Load the CSV data into pandas DataFrame
    data = pd.read_csv(csv_file_path)
    print("CSV data loaded into pandas DataFrame.")

    # Step 4: Convert DataFrame to a list of dictionaries
    # MongoDB prefers documents in dictionary format
    records = data.to_dict('records')
    print("DataFrame converted to list of dictionaries.")

    # Step 5: Insert the records into the MongoDB collection
    if records:
        # Optional: Clear existing data in the collection before inserting
        # mycollection.delete_many({})
        # print(f"Cleared existing data in collection `{collection_name}`.")

        result = mycollection.insert_many(records)
        print(f"Inserted {len(result.inserted_ids)} documents into `{collection_name}`.")
    else:
        print("No data to insert into the collection.")

except Exception as e:
    # Step 6: Handle any errors
    print("Error while connecting to MongoDB or inserting data:", e)

finally:
    # Step 7: Close the MongoDB connection
    if client:
        client.close()
        print("MongoDB connection is closed.")

Accessed database: olistDataNoSQL_teacherbox
Accessed or created collection: product_category
CSV data loaded into pandas DataFrame.
DataFrame converted to list of dictionaries.
Inserted 71 documents into `product_category`.
MongoDB connection is closed.


In [ ]:
# prompt: shape of the above product category df

# The data for `product_category` was loaded into a pandas DataFrame `data` before being inserted into MongoDB.
# We can get the shape of this DataFrame.
print("Shape of the product_category DataFrame:")
data.shape

Shape of the product_category DataFrame:


(71, 2)